# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

**Шаги для выбора локации:**

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Описание данных**  

* id — уникальный идентификатор скважины,
* f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы),
* product — объём запасов в скважине (тыс. баррелей).

**Условия задачи**

* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
* При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
* Бюджет на разработку скважин в регионе — 10 млрд рублей.
* При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
df_0 = pd.read_csv('/datasets/geo_data_0.csv')
df_1 = pd.read_csv('/datasets/geo_data_1.csv')
df_2 = pd.read_csv('/datasets/geo_data_2.csv')

Взглянем на данные с первого региона

In [3]:
df_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
df_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Взглянем на данные со второго региона

In [5]:
df_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [6]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Взглянем на данные с третьего региона

In [7]:
df_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [8]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


*Исходные данные качественные, пропусков нет, тип данных соответствует. Можно сразу переходить к обучению.*

## Обучение и проверка модели

Для каждого месторождения:  
Разобьем данные на обучающую и валидационную выборки в соотношении 75:25. Обучим модель линейной регрессии и сделаем предсказание на валидационной выборке. Затем вычислим средний запас предсказанного сырья и RMSE модели.

In [9]:
# Напишем функцию, которая принимает датасет и возвращает среднее предсказание и RMSE

def model_predictions(data):
    model = LinearRegression()
    features = data.drop(["id","product"], axis = 1)     # id - не нужный признак, его мы удалим
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=42)
    model.fit(features_train, target_train)
    predictions = pd.Series(model.predict(features_valid), index=target_valid.index)
    rmse = mean_squared_error(target_valid, predictions)**0.5
    return target_valid, predictions, rmse

In [10]:
# Получим предсказания и RMSE по каждому месторождению

target_df_0, pred_df_0, rmse_df_0 = model_predictions(df_0)
target_df_1, pred_df_1, rmse_df_1 = model_predictions(df_1)
target_df_2, pred_df_2, rmse_df_2 = model_predictions(df_2)

In [11]:
# Выведем полученные результаты в виде таблицы

results = pd.DataFrame(data=[[pred_df_0.mean(), rmse_df_0], 
                             [pred_df_1.mean(), rmse_df_1], 
                             [pred_df_2.mean(), rmse_df_2]],
                       index=['Регион df_0', 'Регион df_1','Регион df_2'], 
                       columns=['Cредний запас предсказанного сырья', 'RMSE модели'])
results

,Cредний запас предсказанного сырья,RMSE модели
Регион df_0,92.398800,37.756600
Регион df_1,68.712878,0.890280
Регион df_2,94.771024,40.145872


*Из таблицы хорошо видно, что самые большие объемы сырья находятся в нулевом и втором регионах. При этом ошибка модели в данных регионах достаточно высокая.  
Запасы сырья в первом регионе меньше чем в других, но точность модели практически максимальная.*

## Подготовка к расчёту прибыли

* Все ключевые значения для расчётов сохраним в отдельных переменных

In [12]:
#Количество разведываемых точек
PRE_POINTS = 500

#Количество точек для разработки
FINAL_POINTS = 200

#Бюджет на разработку скважин в регионе
BUDGET = 10000000000

#Стоимость тысячи баррелей нефти
PRICE_PER_K_BARREL = 450000

* Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины

In [13]:
NON_DAMAGE_POINT = (BUDGET / PRICE_PER_K_BARREL) / FINAL_POINTS
print('Достаточный объем сырья для безубыточной разработки: {:.2f} тыс. баррелей'.format(NON_DAMAGE_POINT))

Достаточный объем сырья для безубыточной разработки: 111.11 тыс. баррелей


*Рассчитаный нами ранее, средний запас сырья в скважинах, меньше чем точка безубыточности.   
При этом у первого и третьего регионов средние значения запасов сырья более приближены к достаточному объему безубыточной разработки.*

* Напишем функцию для расчёта прибыли

In [14]:
#Стоимость разработки одной скважины
BUDGET_ONE_POINT = BUDGET / FINAL_POINTS

#Функция принимает предсказания по объёму добычи и кол-ву скважин, а вернёт предполагаемую прибыль
def revenue(target, predictions, count):
    top_predictions = predictions.sort_values(ascending=False).head(count)
    return (PRICE_PER_K_BARREL * target.loc[top_predictions.index] - BUDGET_ONE_POINT)

## Расчёт прибыли и рисков 

Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.   
Напишем функцию для определения средней прибыли, 95%-ого доверительного интервала и риска убытков. Риск убытков будем считать как отношение убыточных скважин к общему количеству разрабатываемых скважин.

In [15]:
def bootstrap_1000(target, predictions, point_start=500, point_final=200):
    
    state = np.random.RandomState(43)    
    total_values = []
    minus_values = []
    proba_minus_values = []
    
    for i in range(1000):
        #Выбираем point_start произвольных скважин 
        predictions_sample = predictions.sample(n=point_start, replace=True, random_state=state)
        
        #Считаем прибыль point_final выбранных скважин
        total_revenue = revenue(target, predictions_sample, point_final)
        
        #выбираем из выборки скважины с отрицательной доходностью
        minus_values = total_revenue[total_revenue<=0]
        
        #Считаем вероятность получить убыток
        #proba_minus_revenue = len(minus_values) / len(total_revenue)
        
         #Сохраним прибыль
        total_values.append(total_revenue.sum())
        
        #Coхраним убытки
        #minus_values.append(minus_revenue.sum()) - старый вариант
        
        #Сохраним вероятности убытков
        #proba_minus_values.append(proba_minus_revenue) - старый вариант
        

    #массивы приведем к Series значений в млн. рублей
    total_values = pd.Series(total_values) / 1000000
    minus_values = pd.Series(minus_values) / 1000000
    proba_minus_revenue = len(minus_values) / 1000
    #proba_minus_values = pd.Series(proba_minus_values) - старый вариант
    
    #Cредняя прибыль
    total_mean = total_values.mean() 
    #Нижняя граница доверительного интервала средней прибыли
    lower_total_mean = round(total_values.quantile(.025), 2)
    #Верхняя граница доверительного интервала средней прибыли
    higher_total_mean = round(total_values.quantile(.975), 2)
    
    #Cредний убыток
    minus_values_mean = minus_values.mean()
    #Нижняя граница доверительного интервала среднего убытка
    #lower_minus_values = round(minus_values.quantile(.025), 2)
    #Верхняя граница доверительного интервала среднего убытка
    #higher_minus_values = round(minus_values.quantile(.975), 2)
    
    #Cредняя вероятность получить убыток                              - старый вариант
    #proba_minus_values_mean = proba_minus_values.mean()
    #Нижняя граница доверительного интервала вероятности убытка
    #lower_proba_minus_values = round(proba_minus_values.quantile(.025), 2)
    #Верхняя граница доверительного интервала вероятности убытка
    #higher_proba_minus_values = round(proba_minus_values.quantile(.975), 2) 
    
    return pd.DataFrame(data=[[total_mean], [(lower_total_mean, higher_total_mean)], 
                              [minus_values_mean], 
                              [proba_minus_revenue]], 
                        index=['Средняя прибыль', 'Доверительный интервал средней прибыли', 
                               'Cредний убыток', 
                               'Вероятность убыта'])

Применим нашу функцию к трем данным о месторождениях

In [16]:
region_df_0 = bootstrap_1000(target_df_0, pred_df_0)
region_df_0.columns = ['Регион df_0']
region_df_1 = bootstrap_1000(target_df_1, pred_df_1)
region_df_1.columns = ['Регион df_1']
region_df_2 = bootstrap_1000(target_df_2, pred_df_2)
region_df_2.columns = ['Регион df_2']

In [17]:
#Объеденим получившиеся результаты в едунную общую таблицу
results = results.merge(region_df_2
                        .merge(region_df_1
                               .merge(region_df_0, 
                                      left_index=True, 
                                      right_index=True), 
                               left_index=True, 
                               right_index=True).T, 
                        left_index=True, 
                        right_index=True).T

Посмотрим на результат

In [18]:
results

,Регион df_0,Регион df_1,Регион df_2
Cредний запас предсказанного сырья,92.3988,68.7129,94.771
RMSE модели,37.7566,0.89028,40.1459
Средняя прибыль,420.767,443.471,376.347
Доверительный интервал средней прибыли,"(-128.57, 951.86)","(54.59, 889.33)","(-140.54, 888.49)"
Cредний убыток,-12.1932,-3.59668,-13.508
Вероятность убыта,0.067,0.111,0.086


## Общий вывод

В итоговой таблице видно, что **ни один из регионов не подходит по условию рисков проекта**. Если же выбирать из наименее рискованных, то подходящим будет нулевой регион. При этом набольшая средняя прибыль получилась у первого региона.